# Bonus Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. Use Ironhack's database to load the pokemon data (db: pokemon, table: pokemon_stats). 

In [1]:
# Import libraries
import pandas as pd;
from sqlalchemy import create_engine;

In [2]:
# Database connection parameters
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'pokemon'

# Create the connection
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
query = 'SELECT * FROM pokemon_stats'

# Read the data
pokemon = pd.read_sql(query, engine)

In [3]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [4]:
# Find unique types
unique_types = pokemon['Type 1'].append(pokemon['Type 2']).unique()

# Check number of types
len(unique_types)

19

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [5]:
# If you read the pokemon data using pandas.read_csv, take into account what the hint says 
# and use "if isinstance(ptype, str)" as a condition to skip NaN values. 
# If you read the pokemon data using Ironhack's database, NaN values are empty strings, so 
# use "if ptype != ''" to skip empty values. 

In [6]:
# Create empty list
pokemon_totals = []

# Loop unique types
for ptype in unique_types:
    if ptype != '':
        totals_ptype = pokemon[(pokemon['Type 1']==ptype) | (pokemon['Type 2']==ptype)]['Total']
        pokemon_totals.append(totals_ptype)

# Check number of types
len(pokemon_totals)

18

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [7]:
# The one-way ANOVA tests the null hypothesis that two or 
# more groups have the same population mean.
from scipy.stats import f_oneway

In [8]:
# Ho: the pokemon types have the same population mean.
# Ha: the pokemon types have different populations means. 
st, p = f_oneway(*pokemon_totals)

In [9]:
# Variables 
alpha = 0.05

# Rejection?
print("Null hypothesis rejected") if p < alpha else print("Null hypothesis can't be rejected")
print('Statistic: ', st)
print('p-value: ', p)

Null hypothesis rejected
Statistic:  6.6175382960055344
p-value:  2.6457458815984803e-15


#### Interpret the ANOVA test result. Is the difference significant?

~~~~~
Pokemon types have significantly different population means.
~~~~~